# 2A.ml - Texte et machine learning

Revue de méthodes de [word embedding](https://en.wikipedia.org/wiki/Word_embedding) statistiques (~ [NLP](https://en.wikipedia.org/wiki/Natural_language_processing)) ou comment transformer une information textuelle en vecteurs dans un espace vectoriel (*features*) ? Deux exercices sont ajoutés à la fin.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Données

Nous allons travailler sur des données twitter collectées avec le mot-clé macron : [tweets_macron_sijetaispresident_201609.zip](https://github.com/sdpython/ensae_teaching_cs/raw/master/src/ensae_teaching_cs/data/data_web/tweets_macron_sijetaispresident_201609.zip).

In [2]:
from ensae_teaching_cs.data import twitter_zip
df = twitter_zip(as_df=True)
df.head(n=2).T

,0,1
index,776066992054861825,776067660979245056
nb_user_mentions,0,0
nb_extended_entities,0,0
nb_hashtags,1,1
geo,NaN,NaN
text_hashtags,", SiJétaisPrésident",", SiJétaisPrésident"
annee,2016,2016
delimit_mention,NaN,NaN
lang,fr,fr
id_str,7.76067e+17,7.76068e+17


In [3]:
df.shape

(5088, 20)

5000 tweets n'est pas assez pour tirer des conclusions mais cela donne une idée. On supprime les valeurs manquantes.

In [4]:
data = df[["retweet_count", "text"]].dropna()
data.shape

(5087, 2)

## Construire une pondération

Le texte est toujours délicat à traiter. Il n'est pas toujours évident de sortir d'une information binaire : un mot est-il présent ou pas. Les mots n'ont aucun sens numérique. Une liste de tweets n'a pas beaucoup de sens à part les trier par une autre colonne : les retweet par exemple.

In [5]:
data.sort_values("retweet_count", ascending=False).head()

,retweet_count,text
2038,842.0,#SiJetaisPresident travailler moins pour gagne...
2453,816.0,#SiJetaisPresident je ferais revenir l'été ave...
2627,529.0,#SiJetaisPresident le mcdo livrerai à domicile
1402,289.0,#SiJetaisPresident les devoirs ça serait de re...
2198,276.0,#SiJetaisPresident ? Président c'est pour les...


Sans cette colonne qui mesure la popularité, il faut trouver un moyen d'extraire de l'information. On découpe alors en mots et on constuire un modèle de langage : les [n-grammes](https://fr.wikipedia.org/wiki/N-gramme). Si un tweet est constitué de la séquence de mots $(m_1, m_2, ..., m_k)$. On définit sa probabilité comme :

$$P(tweet) = P(w_1, w_2) P(w_3 | w_2, w_1) P(w_4 | w_3, w_2) ... P(w_k | w_{k-1}, w_{k-2})$$

Dans ce cas, $n=3$ car on suppose que la probabilité d'apparition d'un mot ne dépend que des deux précédents. On estime chaque n-grammes comme suit :

$$P(c | a, b) = \frac{ \# (a, b, c)}{ \# (a, b)}$$

C'est le nombre de fois où on observe la séquence $(a,b,c)$ divisé par le nombre de fois où on observe la séquence $(a,b)$.

### Tokenisation

Découper en mots paraît simple ``tweet.split()`` et puis il y a toujours des surprises avec le texte, la prise en compte des tirets, les majuscules, les espaces en trop. On utilse un *tokenizer* dédié : [TweetTokenizer](http://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.casual.TweetTokenizer) ou un tokenizer qui prend en compte le langage.

In [6]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False)
tokens = tknzr.tokenize(data.loc[0, "text"])
tokens

['#sijétaisprésident',
 'se',
 'serait',
 'la',
 'fin',
 'du',
 'monde',
 '...',
 'mdr',
 '😂']

### n-grammes

* [N-Gram-Based Text Categorization: Categorizing Text With Python](http://blog.alejandronolla.com/2013/05/20/n-gram-based-text-categorization-categorizing-text-with-python/)

In [7]:
from nltk.util import ngrams
generated_ngrams = ngrams(tokens, 4, pad_left=True, pad_right=True)
list(generated_ngrams)

[(None, None, None, '#sijétaisprésident'),
 (None, None, '#sijétaisprésident', 'se'),
 (None, '#sijétaisprésident', 'se', 'serait'),
 ('#sijétaisprésident', 'se', 'serait', 'la'),
 ('se', 'serait', 'la', 'fin'),
 ('serait', 'la', 'fin', 'du'),
 ('la', 'fin', 'du', 'monde'),
 ('fin', 'du', 'monde', '...'),
 ('du', 'monde', '...', 'mdr'),
 ('monde', '...', 'mdr', '😂'),
 ('...', 'mdr', '😂', None),
 ('mdr', '😂', None, None),
 ('😂', None, None, None)]

### Exercice 1 : calculer des n-grammes sur les tweets

### Nettoyage

Tous les modèles sont plus stables sans les stop-words, c'est-à-dire tous les mots présents dans n'importe quel documents et qui n'apporte pas de sens (à, de, le, la, ...). Souvent, on enlève les accents, la ponctuation... Moins de variabilité signifie des statistiques plus fiable.

### Exercice 2 : nettoyer les tweets

Voir [stem](http://www.nltk.org/api/nltk.stem.html#module-nltk.stem).

## Structure de graphe

On cherche cette fois-ci à construire des coordonnées pour chaque tweet.

### matrice d'adjacence

Une option courante est de découper chaque expression en mots puis de créer une matrice *expression x mot* ou chaque case indique la présence d'un mot dans une expression.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
counts = count_vect.fit_transform(data["text"])
counts.shape

(5087, 11924)

On aboutit à une matrice sparse ou chaque expression est représentée à une vecteur ou chaque 1 représente l'appartenance d'un mot à l'ensemble.

In [10]:
type(counts)

scipy.sparse.csr.csr_matrix

In [11]:
counts[:5,:5].toarray()

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]], dtype=int64)

In [12]:
data.loc[0,"text"]

'#SiJétaisPrésident se serait la fin du monde... mdr 😂'

In [13]:
counts[0,:].sum()

8

### td-idf

Ce genre de technique produit des matrices de très grande dimension qu'il faut réduire. On peut enlever les mots rares ou les mots très fréquents. [td-idf](https://fr.wikipedia.org/wiki/TF-IDF) est une technique qui vient des moteurs de recherche. Elle construit le même type de matrice (même dimension) mais associe à chaque couple (document - mot) un poids qui dépend de la fréquence d'un mot globalement et du nombre de documents contenant ce mot.

$$idf(t) = \log \frac{\# D}{\#\{d \; | \; t \in d \}}$$

Où :

* $\#D$ est le nombre de tweets
* $\#\{d \; | \; t \in d \}$ est le nombre de tweets contenant le mot $t$

$f(t,d)$ est le nombre d'occurences d'un mot $t$ dans un document $d$.

$$tf(t,d) = \frac{1}{2} + \frac{1}{2} \frac{f(t,d)}{\max_{t' \in d} f(t',d)}$$

On construit le nombre $tfidf(t,f)$

$$tdidf(t,d) = tf(t,d) idf(t)$$

Le terme $idf(t)$ favorise les mots présent dans peu de documents, le terme $tf(t,f)$ favorise les termes répétés un grand nombre de fois dans le même document. On applique à la matrice précédente.

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
res = tfidf.fit_transform(counts)
res.shape

(5087, 11924)

In [15]:
res[0,:].sum()

2.6988143126521047

### Exercice 3 : tf-idf sans mot-clés

La matrice ainsi créée est de grande dimension. Il faut trouver un moyen de la réduire avec [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

### word2vec

* [word2vec From theory to practice](http://hen-drik.de/pub/Heuer%20-%20word2vec%20-%20From%20theory%20to%20practice.pdf)
* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
* [word2vec](https://radimrehurek.com/gensim/models/word2vec.html)

Cet algorithme part d'une répresentation des mots sous forme de vecteur en un espace de dimension N = le nombre de mots distinct. Un mot est représenté par $(0,0, ..., 0, 1, 0, ..., 0)$. L'astuce consiste à réduire le nombre de dimensions en compressant avec une ACP, un réseau de neurones non linéaires.

In [17]:
sentences = [tknzr.tokenize(_) for _ in data["text"]]
sentences[0]

['#sijétaisprésident',
 'se',
 'serait',
 'la',
 'fin',
 'du',
 'monde',
 '...',
 'mdr',
 '😂']

In [18]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

model = gensim.models.Word2Vec(sentences, min_count=1)

2019-02-19 00:33:17,289 : INFO : collecting all words and their counts
2019-02-19 00:33:17,293 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-02-19 00:33:17,343 : INFO : collected 13262 word types from a corpus of 76468 raw words and 5087 sentences
2019-02-19 00:33:17,344 : INFO : Loading a fresh vocabulary
2019-02-19 00:33:17,453 : INFO : effective_min_count=1 retains 13262 unique words (100% of original 13262, drops 0)
2019-02-19 00:33:17,454 : INFO : effective_min_count=1 leaves 76468 word corpus (100% of original 76468, drops 0)
2019-02-19 00:33:17,535 : INFO : deleting the raw counts dictionary of 13262 items
2019-02-19 00:33:17,536 : INFO : sample=0.001 downsamples 46 most-common words
2019-02-19 00:33:17,538 : INFO : downsampling leaves estimated 56080 word corpus (73.3% of prior 76468)
2019-02-19 00:33:17,597 : INFO : estimated required memory for 13262 words and 100 dimensions: 17240600 bytes
2019-02-19 00:33:17,599 : INFO : resetting layer wei

In [19]:
model.wv.similar_by_word("fin")

2019-02-19 00:33:18,433 : INFO : precomputing L2-norms of word weight vectors


[('ma', 0.9997642636299133),
 ('...', 0.9997348785400391),
 ('via', 0.9997344613075256),
 ('président', 0.9997222423553467),
 ('/', 0.9997204542160034),
 ('mon', 0.9997174143791199),
 ('chaque', 0.9997163414955139),
 ('pays', 0.9997162818908691),
 ('dans', 0.9997103214263916),
 ('ils', 0.9997100830078125)]

In [20]:
model.wv["fin"].shape

(100,)

In [21]:
model.wv["fin"]

array([-1.69395711e-02,  2.17679664e-02, -1.96791530e-01,  4.83000539e-02,
        1.76342070e-01, -1.97501540e-01,  1.16858765e-01,  4.28960919e-02,
       -2.19358385e-01, -6.88297898e-02,  4.69963774e-02,  1.67757738e-02,
        4.84070629e-01,  3.78020816e-02,  4.77045327e-02, -5.80656566e-02,
        1.81520760e-01, -6.32617548e-02, -2.88551152e-01,  1.92677185e-01,
       -1.80626422e-01,  3.46013866e-02, -1.30087599e-01, -1.51333913e-01,
       -3.94276649e-01, -1.74406683e-04, -1.40117511e-01,  2.43928075e-01,
       -1.57584809e-02,  5.71727045e-02, -2.40666732e-01,  1.36840269e-02,
        1.56613309e-02, -1.86894163e-01,  3.16109717e-01,  2.82572895e-01,
       -8.37373435e-02,  5.32252267e-02,  1.00448407e-01, -1.02476187e-01,
       -2.83772196e-03,  2.38204678e-03,  2.23238990e-01, -9.60626006e-02,
        1.96350478e-02,  7.10282177e-02,  4.42795902e-02,  1.33311659e-01,
        8.31384584e-02,  4.17014258e-03,  1.00788459e-01, -1.31932601e-01,
        1.49326697e-01,  

## Tagging

L'objectif est de tagger les mots comme déterminer si un mot est un verbe, un adjectif ...

### grammar

Voir [html.grammar](http://www.nltk.org/api/nltk.html#module-nltk.grammar).

### CRF

Voir [CRF](http://www.nltk.org/api/nltk.tag.html)

### HMM

Voir [HMM](http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.hmm).

## Clustering

Une fois qu'on a des coordonnées, on peut faire plein de choses.

### LDA

* [Latent Dirichlet Application](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
* [LatentDirichletAllocation](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=1000)
tfidf = tfidf_vectorizer.fit_transform(data["text"])

In [23]:
tfidf.shape

(5087, 1000)

In [24]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [25]:
lda.fit(tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [26]:
tf_feature_names = tfidf_vectorizer.get_feature_names()
tf_feature_names[100:103]

['avoir', 'bac', 'bah']

In [27]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [28]:
print_top_words(lda, tf_feature_names, 10)

Topic #0:
gratuit mcdo supprimerai école soir kebab macdo kfc domicile cc
Topic #1:
macron co https de la est le il et hollande
Topic #2:
sijetaispresident je les de la et le des en pour
Topic #3:
notaires eu organiserais mets carte nouveaux journées installation cache créer
Topic #4:
sijetaispresident interdirais les je ballerines la serait serais bah de
Topic #5:
ministre de sijetaispresident la je premier mort et nommerais président
Topic #6:
cours le supprimerais jour sijetaispresident lundi samedi semaine je vendredi
Topic #7:
port interdirait démissionnerais promesses heure rendrai ballerine mes changement christineboutin
Topic #8:
seraient sijetaispresident gratuits aux les nos putain éducation nationale bonne
Topic #9:
bordel seront légaliserai putes gratuites pizza mot virerais vitesse dutreil



### Exercice 4 : LDA

Recommencer en supprimant les stop-words pour avoir des résultats plus propres.